# Wradlib

In [ ]:
import numpy as np
import radarx as rx
import xarray as xr
import xradar as xd
from osgeo import osr
import wradlib as wrl
import cmweather  # noqa
from datetime import datetime

In [ ]:
def filter_radar(ds):
    ds = ds.where((ds.reflectivity >= 0) & (ds.reflectivity <= 70))
    return ds

## Grid Setup

In [ ]:
x_lims = (-300e3, 250e3)
y_lims = (-250e3, 300e3)
z_lims = (0, 15e3)
h_res = 2000
v_res = 500

## Load data

In [ ]:
filename = "KSGF20180612_083109_V06.nc"

In [ ]:
dtree = xd.io.open_cfradial1_datatree(filename)

In [ ]:
dtree = rx.utils.combine_nexrad_sweeps(dtree)

In [ ]:
dtree = dtree.xradar.map_over_sweeps(filter_radar)

In [ ]:
dtree = dtree.xradar.georeference()
dtree.groups

## Gridding

In [ ]:
# Grid setup
sitecoords = (
    dtree["sweep_0"].longitude.values,
    dtree["sweep_0"].latitude.values,
    dtree["sweep_0"].altitude.values,
)

proj = osr.SpatialReference()
proj.ImportFromEPSG(4326)
maxrange = 275e3
maxalt = 20e3
horiz_res = 2000
vert_res = 500
minalt = dtree.altitude.values
minelev = 0.2
maxelev = 21.0

# Create target 3D grid
trgxyz, trgshape = wrl.vpr.make_3d_grid(
    sitecoords, proj, maxrange, maxalt, horiz_res, vert_res
)

print(trgshape)

In [ ]:
tstart = datetime.now()
data_var = "reflectivity"

raw_dt = dtree.xradar.map_over_sweeps(rx.utils.get_geocoords)

proj_crs = xd.georeference.get_crs(raw_dt["sweep_0"].ds)

proj = osr.SpatialReference()

proj.ImportFromEPSG(4326)

swp_list = []
for swp in raw_dt.match("sweep_*"):
    ds = raw_dt[swp].to_dataset()
    xyz = (
        xr.concat(
            [
                ds.coords["x"].reset_coords(drop=True),
                ds.coords["y"].reset_coords(drop=True),
                ds.coords["z"].reset_coords(drop=True),
            ],
            "xyz",
        )
        .stack(npoints=("azimuth", "range"))
        .transpose(..., "xyz")
    )
    swp_list.append(xyz)
xyz = xr.concat(swp_list, "npoints")

data_list = []
for key in list(raw_dt.children):
    if "sweep" in key:
        ds = raw_dt[key].ds
        data = ds[data_var].stack(npoints=("azimuth", "range"))
        data_list.append(data)
data = xr.concat(data_list, "npoints")

# interpolate to Cartesian 3-D volume grid
gridder = wrl.vpr.CAPPI(
    xyz.values,
    trgxyz,
    # gridshape=trgshape,
    maxrange=maxrange,
    minelev=minelev,
    maxelev=maxelev,
)

vol = np.ma.masked_invalid(gridder(data.values).reshape(trgshape))

# diagnostic plot
trgx = trgxyz[:, 0].reshape(trgshape)[0, 0, :]
trgy = trgxyz[:, 1].reshape(trgshape)[0, :, 0]
trgz = trgxyz[:, 2].reshape(trgshape)[:, 0, 0]


lon, lat = rx.utils.cartesian_to_geographic_aeqd(
    trgx,
    trgy,
    data.longitude.values,
    data.latitude.values,
    xd.georeference.get_earth_radius(proj_crs, data.latitude.values),
)

ds_wrl = xr.DataArray(
    data=vol,
    coords={"z": trgz, "y": trgy, "x": trgx},
    dims=("z", "y", "x"),
    name=data_var,
).to_dataset()

ds_wrl["time"] = data.time.mean()
ds_wrl.attrs = dtree.attrs
ds_wrl["latitude"] = data["latitude"]
ds_wrl["longitude"] = data["longitude"]
ds_wrl["lon"] = xr.DataArray(lon, dims=["x"])
ds_wrl["lat"] = xr.DataArray(lat, dims=["y"])
ds_wrl = ds_wrl.set_coords(["lon", "lat"])
print("Wradlib gridding took:", datetime.now() - tstart)
del data, dtree
display(ds_wrl)

## Plot

In [ ]:
ds_wrl.radarx.plot_max_cappi(
    data_var="reflectivity",
    vmin=-10,
    vmax=70,
    range_rings=True,
    add_map=True,
)